In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [215]:
import pandas as pd

ratings_path = "datasets/ratings.csv"
movies_path = "datasets/movies.csv"

ratings = pd.read_csv(ratings_path, sep='\t', encoding='latin-1',
                      usecols=['user_id', 'movie_id', 'user_emb_id', 'movie_emb_id', 'rating'])

movies = pd.read_csv(movies_path, sep='\t', encoding='latin-1',
                     usecols=['movie_id', 'title', 'genres'])

In [216]:
movies.head(5)

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [217]:
ratings.head(5)

,user_id,movie_id,rating,user_emb_id,movie_emb_id
0,1,1193,5,0,1192
1,1,661,3,0,660
2,1,914,3,0,913
3,1,3408,4,0,3407
4,1,2355,5,0,2354


### Content Based

**Objective**: Build a Content-Based Recommendation system that computes similarity between movies based on movie genres. It will suggest movies that are most similar to a particular movie based on its genre.

**Dataset**:
- **Movies Data**: DataFrame with columns `movie_id`, `title`, and `genres`.

**Steps**:

1. **Preprocess Data**:
   - Split the `genres` column into separate genre strings.
   - Fill any missing values in the `genres` column and convert it to string.

2. **Compute TF-IDF Matrix**:
   - Use `TfidfVectorizer` to transform the genres into a TF-IDF matrix.

3. **Calculate Cosine Similarity**:
   - Compute cosine similarity between the TF-IDF matrices of movies.

4. **Recommend Movies**:
   - Create a function that gets movie recommendations based on the cosine similarity score of movie genres.

In [218]:
movies_1 = movies.copy()
ratings_1 = ratings.copy()

In [219]:
movies_1['genres'] = movies_1['genres'].str.split('|')
movies_1['genres'] = movies_1['genres'].fillna("").astype('str')

In [220]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0.0, stop_words='english')
tfidf_matrix = tf.fit_transform(movies_1['genres'])

In [221]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [222]:
def genre_recommendations(title, n=10):
    #TODO
    idx = movies_1.loc[movies['title'].str.lower() == title.lower()].index[0]

    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:n]

    movie_indices = [i[0] for i in sim_scores]
    
    return movies_1['title'].iloc[movie_indices]

    pass

In [223]:
# Example
genre_recommendations('GoldenEye (1995)')

345     Clear and Present Danger (1994)
543           Surviving the Game (1994)
724                    Rock, The (1996)
788                     Daylight (1996)
825               Chain Reaction (1996)
978                 Maximum Risk (1996)
1467                    Anaconda (1997)
1513                     Con Air (1997)
1693                   Firestorm (1998)
Name: title, dtype: object


### Collaborative Filtering

**Objective**: Implement a collaborative filtering movie recommendation system to recommend movies to a user based on similar users' spreferences.

**Dataset**:

- **Ratings Data**: DataFrame with columns `user_id`, `movie_id`, and `rating`.
- **Movies Data**: DataFrame with columns `movie_id`, `title`, and `genres`.

**Steps**:

1. **Preprocess Data**:
   - Merge `ratings` and `movies` DataFrames on `movie_id`.
   - Split `genres` column into separate rows.

2. **Create User-Genre Matrix**:
   - Create a matrix where rows are users and columns are genres.
   - Each cell represents the ratio of movies watched by the user in that genre to the total movies watched by the user.

3. **Calculate User Similarity**:
   - Use cosine similarity to calculate the similarity between users.

4. **Find Top N Similar Users**:
   - Find the top N users with the most similar preferences to the given user.

5. **Recommend Movies**:
   - Recommend movies that similar users have watched but the target user has not.
   - Return titles and genres of the top 10 recommended movies.

In [224]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [225]:
data_2 = pd.merge(ratings, movies, on='movie_id')

In [226]:
data_2.head(5)

,user_id,movie_id,rating,user_emb_id,movie_emb_id,title,genres
0,1,1193,5,0,1192,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,0,660,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,0,913,My Fair Lady (1964),Musical|Romance
3,1,3408,4,0,3407,Erin Brockovich (2000),Drama
4,1,2355,5,0,2354,"Bug's Life, A (1998)",Animation|Children's|Comedy


In [227]:
data_2 = data_2.assign(genres=data_2['genres'].str.split('|')).explode('genres')

In [228]:
data_2.head(5)

,user_id,movie_id,rating,user_emb_id,movie_emb_id,title,genres
0,1,1193,5,0,1192,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,0,660,James and the Giant Peach (1996),Animation
1,1,661,3,0,660,James and the Giant Peach (1996),Children's
1,1,661,3,0,660,James and the Giant Peach (1996),Musical
2,1,914,3,0,913,My Fair Lady (1964),Musical


In [229]:
print(len(data_2['genres'].unique()), len(data_2['user_id'].unique()))

18 6040


In [233]:
user_movie_counts = data_2.groupby('user_id').size().reset_index(name='total_movies')
user_movie_counts.head()

,user_id,total_movies
0,1,116
1,2,288
2,3,123
3,4,58
4,5,352


In [234]:
user_genre_counts = data_2.groupby(['user_id', 'genres']).size().reset_index(name='genre_count')
user_genre_counts.head()

,user_id,genres,genre_count
0,1,Action,5
1,1,Adventure,5
2,1,Animation,18
3,1,Children's,20
4,1,Comedy,14


In [235]:
user_genre = pd.merge(user_genre_counts, user_movie_counts, on='user_id')
user_genre['genre_proportion'] = user_genre['genre_count'] / user_genre['total_movies']
user_genre_matrix = user_genre.pivot(index='user_id', columns='genres', values='genre_proportion').fillna(0)
user_genre_matrix.head()

genres,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
user_id,,,,,,,,,,,,,,,,,,
1,0.043103,0.043103,0.155172,0.172414,0.120690,0.017241,0.000000,0.181034,0.025862,0.000000,0.000000,0.120690,0.000000,0.051724,0.025862,0.025862,0.017241,0.000000
2,0.194444,0.065972,0.000000,0.000000,0.086806,0.041667,0.000000,0.274306,0.003472,0.003472,0.006944,0.000000,0.010417,0.083333,0.059028,0.107639,0.052083,0.010417
3,0.186992,0.203252,0.024390,0.024390,0.243902,0.000000,0.000000,0.065041,0.016260,0.000000,0.024390,0.008130,0.008130,0.040650,0.048780,0.040650,0.016260,0.048780
4,0.327586,0.103448,0.000000,0.017241,0.000000,0.017241,0.000000,0.103448,0.034483,0.000000,0.051724,0.000000,0.000000,0.034483,0.155172,0.068966,0.051724,0.034483
5,0.088068,0.025568,0.011364,0.017045,0.159091,0.059659,0.017045,0.295455,0.000000,0.008523,0.028409,0.008523,0.022727,0.085227,0.042614,0.110795,0.017045,0.002841


In [236]:
def get_top_n_similar_users(user_id, n=5):
    #TODO
    target_user_vector = user_genre_matrix.loc[user_id].values.reshape(1, -1)
    # print(target_user_vector)
    similarities = cosine_similarity(user_genre_matrix, target_user_vector)
    similar_users = similarities.flatten().argsort()[::-1][1:n+1]
    # print(similar_users + 1)
    return similar_users + 1
    pass

In [237]:
def recommend_movies(user_id, n_similar_users=5, n_recommendations=10):
    #TODO

    similar_users = get_top_n_similar_users(user_id, n_similar_users)
    similar_users_movies = data_2[data_2['user_id'].isin(similar_users)]
    target_user_movies = data_2[data_2['user_id'] == user_id]['movie_id'].unique()
    recommendations = similar_users_movies[~similar_users_movies['movie_id'].isin(target_user_movies)]
    movie_recommendations = recommendations.groupby('movie_id').size().reset_index(name='count')
    # print(movie_recommendations.head(10))
    top_movie_recommendations = movie_recommendations.sort_values(by='count', ascending=False).head(n_recommendations)
    # print(top_movie_recommendations.head(10))
    top_movies = pd.merge(top_movie_recommendations, movies, on='movie_id')[['movie_id', 'title', 'genres']]
    
    return top_movies.sort_values(by="movie_id")


    pass

In [238]:
# Example
user_id = 1
recommended_movies = recommend_movies(user_id, n_similar_users=5, n_recommendations=10)

In [239]:
recommended_movies

,movie_id,title,genres
8,34,Babe (1995),Children's|Comedy|Drama
9,912,Casablanca (1942),Drama|Romance|War
3,1032,Alice in Wonderland (1951),Animation|Children's|Musical
7,1282,Fantasia (1940),Animation|Children's|Musical
2,1617,L.A. Confidential (1997),Crime|Film-Noir|Mystery|Thriller
4,2078,"Jungle Book, The (1967)",Animation|Children's|Comedy|Musical
1,2080,Lady and the Tramp (1955),Animation|Children's|Comedy|Musical|Romance
0,2081,"Little Mermaid, The (1989)",Animation|Children's|Comedy|Musical|Romance
5,2087,Peter Pan (1953),Animation|Children's|Fantasy|Musical
6,2096,Sleeping Beauty (1959),Animation|Children's|Musical


### SVD (Singular Value Decomposition)



**Objective**: Implement an SVD-based recommendation system to recommend movies to users by decomposing the user-item interaction matrix into latent factors.

**Dataset**:
- **Ratings Data**: DataFrame with columns `user_id`, `movie_id`, and `rating`.
- **Movies Data**: DataFrame with columns `movie_id`, `title`, and `genres`.

**Steps**:

1. **Preprocess Data**:
   - Merge `ratings` and `movies` DataFrames on `movie_id`.
   - Create a user-item matrix where rows represent users, columns represent movies, and the values are the ratings.

2. **Decompose Matrix using SVD**:
   - Apply Singular Value Decomposition (SVD) to decompose the user-item matrix into three matrices: $U$, $\Sigma$, and $V^T$.

3. **Reconstruct Matrix**:
   - Reconstruct the user-item matrix using the top $k$ singular values to reduce dimensionality.

4. **Predict Ratings**:
   - Use the reconstructed matrix to predict ratings for all user-item pairs.

5. **Recommend Movies**:
   - Recommend the top 20 movies with the highest predicted ratings for a given user that the user hasn't rated yet.

In [240]:
n_users = ratings.user_id.unique().shape[0]
n_movies = ratings.movie_id.unique().shape[0]
print('Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_movies))

Number of users = 6040 | Number of movies = 3706


Fill na elements.

In [241]:
Ratings_3 = ratings.pivot(index = 'user_id', columns ='movie_id', values = 'rating').fillna(0)
Ratings_3.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [254]:
# calculate rating matrix
from scipy.sparse.linalg import svds
# user_ratings_mean = np.mean(Ratings_3, axis=1)
# Ratings_demeaned = Ratings_3.sub(user_ratings_mean, axis=0)
rating_matrix = Ratings_3.to_numpy()

In [261]:
U, sigma, Vt = svds(rating_matrix, k=50)
sigma = np.diag(sigma)

In [262]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt)

In [263]:
preds = pd.DataFrame(all_user_predicted_ratings, columns=Ratings_3.columns)
preds.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
0,4.292556,0.164772,-0.184504,-0.018528,0.021516,-0.181793,-0.102936,0.157111,-0.058523,-0.164231,...,0.032475,0.007048,0.033170,-0.007577,-0.075329,0.394578,0.124714,0.051545,0.057349,0.076743
1,0.754817,0.128981,0.341128,0.009540,0.001830,1.314580,0.077427,0.062218,0.163983,1.514572,...,-0.051376,-0.015780,-0.010918,0.055628,-0.016037,0.166941,-0.421144,-0.106675,-0.049650,-0.126564
2,1.844858,0.473855,0.098573,-0.039309,-0.019895,-0.154038,-0.141531,0.111551,0.035977,0.738427,...,0.049994,0.003596,0.022734,0.043377,0.034825,0.120205,0.086553,0.034783,0.029337,-0.121871
3,0.395484,-0.045487,0.033716,0.083890,0.051561,0.260676,-0.081541,0.023891,0.051932,-0.077170,...,0.010847,0.007518,0.004269,0.009527,-0.079630,0.077845,0.051369,-0.017046,0.018980,-0.047580
4,1.557609,-0.006700,-0.044860,0.249014,-0.043320,1.518140,-0.164306,-0.043626,-0.078771,0.424064,...,0.097708,0.013847,-0.024256,-0.043126,-0.065997,-0.025571,0.517685,0.008508,0.106630,0.221219


In [264]:
def recommend_movies(predictions, userID, movies, original_ratings, num_recommendations):

    user_row_number = userID - 1 # User ID starts at 1, not 0
    sorted_user_predictions = predictions.iloc[user_row_number].sort_values(ascending=False)

    user_data = original_ratings[original_ratings.user_id == userID]

    # merfe movies data with user_data
    user_full = user_data.merge(movies, how="left", left_on="movie_id", right_on="movie_id").sort_values(["rating"], ascending=False)


    print(f'User {userID} has already rated {user_full.shape[0]} movies.')
    print(f'Recommending highest {num_recommendations} predicted ratings movies not already rated.')

    recommendations = (movies[~movies["movie_id"].isin(user_full["movie_id"])].
                       merge(pd.DataFrame(sorted_user_predictions).reset_index(), how="left", left_on="movie_id", right_on="movie_id").
                       rename(columns={user_row_number: "Predictions"}).
                       sort_values("Predictions", ascending=False).
                       iloc[:num_recommendations, :-1])

    return user_full, recommendations

In [266]:
already_rated, predictions = recommend_movies(preds, 4375, movies, ratings, 20)

User 4375 has already rated 325 movies.
Recommending highest 20 predicted ratings movies not already rated.


In [267]:
ratings.head(20)

,user_id,movie_id,rating,user_emb_id,movie_emb_id
0,1,1193,5,0,1192
1,1,661,3,0,660
2,1,914,3,0,913
3,1,3408,4,0,3407
4,1,2355,5,0,2354
5,1,1197,3,0,1196
6,1,1287,5,0,1286
7,1,2804,5,0,2803
8,1,594,4,0,593
9,1,919,4,0,918


In [269]:
# Top 20 movies that User 4375 has rated
already_rated.head(20)

,user_id,movie_id,rating,user_emb_id,movie_emb_id,title,genres
195,4375,3250,5,4374,3249,Alive (1993),Drama
285,4375,175,5,4374,174,Kids (1995),Drama
199,4375,3257,5,4374,3256,"Bodyguard, The (1992)",Action|Drama|Romance|Thriller
65,4375,3809,5,4374,3808,What About Bob? (1991),Comedy
210,4375,2688,5,4374,2687,"General's Daughter, The (1999)",Drama|Thriller
270,4375,150,5,4374,149,Apollo 13 (1995),Drama
271,4375,2710,5,4374,2709,"Blair Witch Project, The (1999)",Horror
125,4375,349,5,4374,348,Clear and Present Danger (1994),Action|Adventure|Thriller
60,4375,288,5,4374,287,Natural Born Killers (1994),Action|Thriller
141,4375,47,5,4374,46,Seven (Se7en) (1995),Crime|Thriller


#### Evaluation

In [270]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

train_data, test_data = train_test_split(ratings, test_size=0.2, random_state=42)

train_ratings = train_data.pivot(index="user_id", columns="movie_id", values="rating").fillna(0)
R_train = train_ratings.values

user_ratings_mean_train = np.mean(R_train, axis=1)
Ratings_demeaned_train = R_train - user_ratings_mean_train.reshape(-1, 1)

U_train, sigma_train, Vt_train = svds(Ratings_demeaned_train, k=50)
sigma_train = np.diag(sigma_train)

all_user_predicted_ratings_train = np.dot(np.dot(U_train, sigma_train), Vt_train) + user_ratings_mean_train.reshape(-1, 1)
preds_train = pd.DataFrame(all_user_predicted_ratings_train, columns=train_ratings.columns)

def predict_rating(user_id, movie_id):
    try:
        pred = preds_train.loc[user_id -1, movie_id]
    except KeyError:
        pred = np.nan
    return pred

test_data['predicted_rating'] = test_data.apply(lambda x: predict_rating(x['user_id'], x['movie_id']), axis=1)

test_data.dropna(inplace=True)

rmse = sqrt(mean_squared_error(test_data['rating'], test_data['predicted_rating']))
print(f'Root Mean Squared Error: {rmse}')


Root Mean Squared Error: 2.7404660761407365


### Nueral Network Model (Recommender Model)



**Objective**: Implement a Recommender model to recommend movies to a user based on similar users' preferences.

**Dataset**:
- **Ratings Data**: DataFrame with columns `user_id`, `movie_id`, and `rating`.
- **Movies Data**: DataFrame with columns `movie_id`, `title`, and `genres`.

**Steps**:

1. **Define Dataset and DataLoader**:
   - Create a custom PyTorch `Dataset` for ratings.
   - Create a DataLoader for batching and shuffling data.

2. **Define the Neural Network**:
   - Create a neural network with embedding layers for users and movies.

3. **Train the Model**:
   - Train the model using Mean Squared Error loss and Adam optimizer.
   - Save model checkpoints.

4. **Evaluate the Model**:
   - Calculate RMSE on the entire dataset.

5. **Predict Ratings for Unrated Movies**:
   - Predict and recommend top 10 unrated movies for a given user.

In [272]:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from tqdm import tqdm

In [315]:
# Define the dataset
class RatingsDataset(Dataset):
    def __init__(self, ratings):
        #TODO
        self.user_ids = torch.tensor(ratings["user_id"].values, dtype=torch.long)
        self.movie_ids = torch.tensor(ratings["movie_id"].values, dtype=torch.long)
        self.ratings = torch.tensor(ratings["rating"].values, dtype=torch.float32)

    def __len__(self):
        return len(self.user_ids)
        
    def __getitem__(self, idx):
        return self.user_ids[idx], self.movie_ids[idx], self.ratings[idx]

# Define the neural network
class RecommenderNet(nn.Module):
    def __init__(self, num_users, num_movies, embedding_size=100):
        super(RecommenderNet, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_size)
        self.movie_embedding = nn.Embedding(num_movies, embedding_size)
        self.fc_layeres = nn.Sequential(
            nn.Linear(embedding_size * 2, 100),
            nn.ReLU(),
            nn.Linear(100, 50),
            nn.ReLU(),
            nn.Linear(50, 1)
        )
        self.bn = nn.BatchNorm1d(embedding_size * 2)
        

    def forward(self, user, movie):
        # print(user, movie)
        # print(f"user shape: {user.shape}, movie shape: {movie.shape}")
        user_embedding = self.user_embedding(user)
        movie_embedding = self.movie_embedding(movie)
        concatenated = torch.cat((user_embedding, movie_embedding), dim=1)
        concatenated = self.bn(concatenated)
        output = self.fc_layeres(concatenated)
        return output.view(-1)

# Create the dataset and dataloader
dataset = RatingsDataset(ratings)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# Initialize the model, loss function and optimizer
num_users = ratings["user_id"].max()
num_movies = ratings["movie_id"].max()


print(f"num_users : {num_users}, num_movies : {num_movies}")
model = RecommenderNet(num_users + 1, num_movies + 1)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# Training loop
num_epochs = 5
for epoch in tqdm(range(num_epochs), desc="Training"):
    model.train()
    running_loss = 0.0
    for user, movie, rating in tqdm(dataloader, desc="epoch"):
        optimizer.zero_grad()
        # print(f"user: {user}, movie: {movie}")
        output = model(user, movie)
        loss = criterion(output, rating)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss/len(dataloader)}")

torch.save(model.state_dict(), "recommender_model.pth")
# Evaluation
model.eval()
predictions = []
targets = []
with torch.no_grad():
    for user, movie, rating in dataloader:
        output = model(user, movie)
        predictions.extend(output.numpy())
        targets.extend(rating.numpy())
predictions = np.array(predictions)
targets = np.array(targets)
rmse = np.sqrt(np.mean((predictions - targets) ** 2))
print(f"RMSE on the entire dataset: {rmse}")

num_users : 6040, num_movies : 3952


Training:  20%|██        | 1/5 [02:45<11:03, 165.93s/it]

Epoch 1/5, Loss: 0.9524236575690987


Training:  40%|████      | 2/5 [04:40<06:47, 135.81s/it]

Epoch 2/5, Loss: 0.8443994396547033


...

Epoch 3/5, Loss: 0.818642117273282


Training:  80%|████████  | 4/5 [08:14<01:56, 116.11s/it]

Epoch 4/5, Loss: 0.7993530265749038


Training: 100%|██████████| 5/5 [10:01<00:00, 120.30s/it]


Epoch 5/5, Loss: 0.7816663714763001
RMSE on the entire dataset: 0.8627090454101562


In [316]:
# Example

sample_user_id = 1
sample_user_data = ratings[ratings['user_id'] == sample_user_id]
sample_dataset = RatingsDataset(sample_user_data)
sample_dataloader = DataLoader(sample_dataset, batch_size=1, shuffle=False)

print(f'Sample predictions for user ID {sample_user_id}:')
with torch.no_grad():
    for user, movie, rating in sample_dataloader:
        output = model(user, movie).squeeze()
        print(f'Movie ID: {movie.item()}, Predicted Rating: {output.item()}, Actual Rating: {rating.item()}')

Sample predictions for user ID 1:
Movie ID: 1193, Predicted Rating: 4.544903755187988, Actual Rating: 5.0
Movie ID: 661, Predicted Rating: 3.4655418395996094, Actual Rating: 3.0
Movie ID: 914, Predicted Rating: 4.101353645324707, Actual Rating: 3.0
Movie ID: 3408, Predicted Rating: 3.973872423171997, Actual Rating: 4.0
Movie ID: 2355, Predicted Rating: 3.8438265323638916, Actual Rating: 5.0
Movie ID: 1197, Predicted Rating: 4.454877853393555, Actual Rating: 3.0
Movie ID: 1287, Predicted Rating: 4.259732246398926, Actual Rating: 5.0
Movie ID: 2804, Predicted Rating: 4.310150146484375, Actual Rating: 5.0
Movie ID: 594, Predicted Rating: 3.7831130027770996, Actual Rating: 4.0
Movie ID: 919, Predicted Rating: 4.357565879821777, Actual Rating: 4.0
Movie ID: 595, Predicted Rating: 3.9821393489837646, Actual Rating: 5.0
Movie ID: 938, Predicted Rating: 3.7298007011413574, Actual Rating: 4.0
Movie ID: 2398, Predicted Rating: 4.16249942779541, Actual Rating: 4.0
Movie ID: 2918, Predicted Rating

In [317]:
def predict_unrated_movies(user_id, model, ratings, movies):
    
    rated_movies = ratings[ratings["user_id"] == user_id]["movie_id"].tolist()
    all_movies = torch.tensor(movies["movie_id"].values, dtype=torch.long)
    unrated_movies = torch.tensor([movie_id for movie_id in all_movies if movie_id not in rated_movies], dtype=torch.long)

    user_ids = torch.tensor([user_id] * len(unrated_movies), dtype=torch.long)

    model.eval()
    with torch.no_grad():
        predictions = model(user_ids, unrated_movies)

    _, top_indices = torch.topk(predictions, k=10)
    top_movie_ids = unrated_movies[top_indices].numpy()
    recommended_movies = movies[movies["movie_id"].isin(top_movie_ids)][["movie_id" ,"title", "genres"]]
    recommended_movies["predicted_rating"] = predictions[top_indices].numpy()
    return recommended_movies


# Example
user_id = 1
predictions = predict_unrated_movies(user_id, model, ratings, movies)
print(predictions)

      movie_id                                              title  \
315        318                   Shawshank Redemption, The (1994)   
892        904                                 Rear Window (1954)   
896        908                          North by Northwest (1959)   
910        922      Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)   
1162      1178                              Paths of Glory (1957)   
1180      1198                     Raiders of the Lost Ark (1981)   
1194      1212                              Third Man, The (1949)   
1950      2019  Seven Samurai (The Magnificent Seven) (Shichin...   
2434      2503                            Apple, The (Sib) (1998)   
3269      3338                             For All Mankind (1989)   

                genres  predicted_rating  
315              Drama          4.880655  
892   Mystery|Thriller          4.870305  
896     Drama|Thriller          4.812331  
910          Film-Noir          4.762222  
1162         Drama|War   

### GMM (Gaussian Mixture Model)

**Objective**: Use a Gaussian Mixture Model to analyze and cluster the click data based on the number of clicks from different locations, aiming to identify distinct patterns of user behavior across 10 countries.

**Dataset**:
- **Click Data**: DataFrame with columns `link_id`, `location`, and `number_of_clicks`.

**Steps**:

1. **Preprocess Data**:
   - Ensure the dataset contains 10 distinct countries.
   - Create a matrix where rows represent different links and columns represent the number of clicks from each country.
   - Normalize the number of clicks to account for different scales.

2. **Fit GMM**:
   - Apply a Gaussian Mixture Model (GMM) to the click data matrix to identify clusters of links with similar click patterns across different countries.

3. **Evaluate Model**:
   - Analyze the resulting model by calculating the log lokelihood, BIC and AIC metrics.


In [320]:
import pandas as pd
import numpy as np
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt

file_path = "datasets/gmm-dataset.csv"
df = pd.read_csv(file_path)

df.head()

,URL,United States,China,India,Japan,Brazil,Russia,Indonesia,Germany,United Kingdom,France
0,url_1,657,194,310,790,768,436,189,973,448,335
1,url_2,389,149,354,729,629,937,345,967,764,104
2,url_3,979,475,509,769,270,705,113,931,977,213
3,url_4,524,699,692,694,473,160,484,37,956,899
4,url_5,767,661,319,572,399,401,277,596,631,203


In [323]:
urls = df['URL']
clicks_data = df.drop('URL', axis=1)
print(f"number of countries : {clicks_data.shape[1]}")

number of countries : 10


In [324]:
from sklearn.preprocessing import StandardScaler
clicks_matrix = clicks_data.values
sclaer = StandardScaler()
clicks_matrix_normalized = sclaer.fit_transform(clicks_matrix)

In [331]:
gmm = GaussianMixture(n_components=10, random_state=42)
gmm.fit(clicks_matrix_normalized)
cluster_labels = gmm.predict(clicks_matrix_normalized)

log_likelihood = gmm.score(clicks_matrix_normalized)
bic = gmm.bic(clicks_matrix_normalized)
aic = gmm.aic(clicks_matrix_normalized)

print(f"Log-Likelihood: {log_likelihood}")
print(f"BIC: {bic}")
print(f"AIC: {aic}")

Log-Likelihood: -1.465031663827093
BIC: 3327.8134853315714
AIC: 1611.0063327654186
